## Fashion MINIST Layer

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.1.0
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [2]:
# # 使用main()函数，避免全局变量的污染，在文件作为执行文件时，首先执行 main() 函数，而对于import该
# # 模块时，则不执行main()函数
# def main():
#     pass

# if __name__ == '__main__':
#     main()

In [3]:
# 加载数据集
(x, y), (x_test, y_test) = datasets.fashion_mnist.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [4]:
# 编写数据集预处理函数，此处接受的 x ，y 已经是tensor
def preprocess(x, y):
    # 把x转换为 tf.float32类型，并压缩到 [0, 1] 之间，注意：此处除以 255. (包括小数点)
    x = tf.cast(x, dtype=tf.float32) / 255.
    # 只需把y转换为 tf.int32即可
    y = tf.cast(y, dtype=tf.int32)
    pass
    # 
    
    return x, y

In [5]:
# 对训练数据（x，y）构建切片数据集
db = tf.data.Dataset.from_tensor_slices((x, y))
# 对构建的数据集db 进行预处理，使用映射函数 map(),注意：map()函数中使用函数名作为参数，不带括号
# 对数据使用shuffle(10000) 处理，以此打乱数据集,shuffle函数中传入一个较大的量，如10000
# 对数据进行划分batch（批次），提高处理数据效率，首先指定batch的大小，定义batchsz变量
batchsz = 128
db = db.map(preprocess).shuffle(10000).batch(batchsz)

# 类似地，对测试数据（x_test，y_test）构建切片数据集，并且把数据转换为tensor类型
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
# 对于测试集来说，不需要调用shuffle函数对测试集打乱处理
db_test = db_test.map(preprocess).batch(batchsz)

# 打印sample，显示效果
# 对可迭代对象db 使用iter()构建迭代器，并使用 next() 返回迭代器的下一个值
db_iter = iter(db)
sample = next(db_iter)
# sample 是迭代器返回的每个值：[x, y]，其中sample[0]指一个batch的x，sample[1]指一个batch的y
print('batch : ', sample[0].shape, sample[1].shape)

batch :  (128, 28, 28) (128,)


In [6]:
'''
创建全连接神经网络(Dense)，命名为 model ，使用 tf.keras.Sequential([]),创建5层的NeuralNetwork， 
Sequential是一个神经网络容器，该 Sequential容器接受一包含神经网络层的列表，以此定义一个人工神经网络，
此处命名为 model
''' 
model = Sequential([
    # 第一层，是全连接层，调用tf.keras.layers.Dense()方法，该方法接受第一参数指明这一层的输出维度数，
    # 其中输入维度默认由TensorFlow推算生成，然后使用activation参数指定选用的激活函数
    # 按照降维的情况选择各层合适的输出维度数
    # 注意，此处是一个列表，应该在每个元素之间添加 逗号
    layers.Dense(256, activation=tf.nn.relu),    # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),    # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),     # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),     # [b, 64] => [b, 32]
    # 输出层一般不需要激活函数
    layers.Dense(10)                             # [b, 32] => [b, 10]
])

# 把训练数据feed 给神经网络(model),根据训练数据的维度，构建各层连接权值，input_shape参数指定维度数
model.build(input_shape=[None, 28*28])
# 调用model的 summary() 方法调试神经网络，该方法可以打印神经网络结构，各层信息，包括参数的个数
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  32896     
_________________________________________________________________
dense_2 (Dense)              multiple                  8256      
_________________________________________________________________
dense_3 (Dense)              multiple                  2080      
_________________________________________________________________
dense_4 (Dense)              multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


通过上面代码的summary()方法，可以看到，总参数量为244,522  其中每个参数占用4个Byte。进而可以算出占用内存量

In [7]:
# 数据前向传播

# 定义优化器，其中参数 lr 表示学习率:1e-3,表示0.001。优化器实现对每一个参数（层之间的链接权重）更新。
# 实现 w = w - lr*grad，b = b - lr*grad , 在调用优化器时，只需传入需要更新的参数列表，
# 完成对参数列表中所有的参数进行更新
optimizer = optimizers.Adam(lr=1e-3)   #*********
# 定义 epoch ，每一epoch 一次数据的迭代
for epoch in range(30):
    # 每步训练db中的一个batch，调用枚举函数enumerate(),每次获得带有索引的一个batch的训练数据
    for step, (x, y) in enumerate(db):
        # x: [b, 28, 28] => [b,28*28]  ;   y: [b]
        x = tf.reshape(x, [-1, 28*28])
        # 求解梯度,使用with...as 语句,调用tf.GradientTape()
        with tf.GradientTape() as tape:
            # 前向传播，仅需调用medel(),(其实质的调用model的call()函数),给model()feed训练数据x即可
            # [b, 784] => [b, 10]
            # 内部调用为  model.__call__(x),若在自定义网络层中，如果实现了call()方法，则优先调用call()
            logits = model(x) # 内部调用为  model.__call__(x),若在自定义网络层中，如果实现了call()方法
            # 对训练数据中的 y 进行 one_hot 编码，便于计算前向传播损失
            y_onehot = tf.one_hot(y, depth=10)
            # 求解损失loss
            # tf.losses.MSE() 计算均方差损失，MSE()的第一个参数是真是分布（训练数据），第二个参数是预测
            # 分布，返回每一个实例的均方差，shape is [b]，然后再使用tf.reduce_mean()，求得所有实例的均值
            loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot, logits))
            # 使用另一种计算loss的方法——交叉熵误差，调用categorical_crossentropy()方法，其中使用
            # from_logits=True表示预测数据是直接从神经网络中输出的原始数据，未经过其他处理，达到数据稳当
            # 然后再使用tf.reduce_mean()，求得所有实例的均值(scalar)
            loss_ce = tf.reduce_mean(tf.losses.categorical_crossentropy(
                y_onehot, logits, from_logits=True))    #*********
            
        # 计算loss对model中参数（链接权重）的梯度
        # 其中使用 model的属性——trainable_variables，返回model的参数列表[w1, b1, w2, b2, .., w5, b5]
        grads = tape.gradient(loss_ce, model.trainable_variables)
        # 调用优化器optimizer的apply_gradients()方法，使用梯度对所有参数trainable_variables（链接权重）
        #进行一次原地更新，其中zip(grads,model.trainable_variables)表示把grads与trainable_variables对应
        # 起来？？？完成更新
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        # step每增加100对结果进行打印显示
#         if step % 100 == 0:
#             print(epoch, step, 'loss_ce: ', float(loss_ce), 'loss_mse: ', float(loss_mse))
            
        # 测试,不需要计算梯度等，同时测试也只进行一个 epoch 即可，不需要设定 batch
        
    # 定义统计正确匹配的数的变量
    total_correct = 0
    # 定义总数变量
    total_num = 0
    for x, y in db_test:
        x = tf.reshape(x, [-1, 28*28])
        logits = model(x)  # [b, 10]
        # logits => probability， 使用softmax()函数，把logits值压缩到[0, 1]之间的等效概率值
        # axis=1 表示在第二个维度上进行转换
        prob = tf.nn.softmax(logits, axis=1)
        # prop => dredict,把prob中概率最大的数转换为位置,调用tf.argmax()函数
        # [b, 10] => [b]   此时获得的是 tf.int64 类型的值，需要进行类型转化
        pred = tf.cast(tf.argmax(prob, axis=1),tf.int32)
        # 拿 pred 值与测试数据y_test 进行比较
        # pred:    [b]
        # y:       [b]
        # correct: [b]   True or False
        correct = tf.equal(pred, y)
        # 把correct中的 boolean 类型转换为int32类型,然后进行求和操作，获得正确匹配的个数
        correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))
            
        total_correct += int(correct)   # 使用int() 函数把tensor类型转化为numpy的int类型
        total_num += x.shape[0]    # x.shape[0] 是测试集中测试样本的个数
            
    # 计算正确率 acc
    acc = total_correct  * 100 / total_num
    print(epoch,'test acc: ',acc,'%')
        
         

0 test acc:  81.28 %
1 test acc:  83.91 %
2 test acc:  84.93 %
3 test acc:  85.39 %
4 test acc:  85.72999999999999 %
5 test acc:  86.1 %
6 test acc:  86.33999999999999 %
7 test acc:  86.50999999999999 %
8 test acc:  86.79 %
9 test acc:  86.94 %
10 test acc:  87.08 %
11 test acc:  87.33999999999999 %
12 test acc:  87.42 %
13 test acc:  87.64 %
14 test acc:  87.6 %
15 test acc:  87.64 %
16 test acc:  87.76 %
17 test acc:  87.79 %
18 test acc:  87.81 %
19 test acc:  87.99 %
20 test acc:  87.99 %
21 test acc:  88.09 %
22 test acc:  88.02 %
23 test acc:  88.14999999999999 %
24 test acc:  88.14 %
25 test acc:  88.25 %
26 test acc:  88.27000000000001 %
27 test acc:  88.28 %
28 test acc:  88.26 %
29 test acc:  88.3 %
